In [1]:
import pandas as pd
from sqlalchemy import create_engine
import getpass

# Obtenemos la contraseña de forma segura.
password = getpass.getpass()

# Creamos la cadena de conexión.
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)

- Creamos la función rentals_month
- Esta función recuperará los datos de alquiler para un mes y año específicos.

In [5]:
def rentals_month(engine, month, year):
    query = text(f"""
    SELECT rental_id, rental_date, customer_id
    FROM rental
    WHERE MONTH(rental_date) = :month AND YEAR(rental_date) = :year;
    """)
    
    with engine.connect() as connection:
        result = connection.execute(query, {"month": month, "year": year})
        df = pd.DataFrame(result.fetchall(), columns=result.keys())
    return df

# Ejemplo:
df_may = rentals_month(engine, 5, 2005)
df_june = rentals_month(engine, 6, 2005)

In [6]:
df_may

,rental_id,rental_date,customer_id
0,1,2005-05-24 22:53:30,130
1,2,2005-05-24 22:54:33,459
2,3,2005-05-24 23:03:39,408
3,4,2005-05-24 23:04:41,333
4,5,2005-05-24 23:05:21,222
...,...,...,...
1151,1153,2005-05-31 21:36:44,506
1152,1154,2005-05-31 21:42:09,59
1153,1155,2005-05-31 22:17:11,251
1154,1156,2005-05-31 22:37:34,106


In [7]:
df_june

,rental_id,rental_date,customer_id
0,1158,2005-06-14 22:53:33,416
1,1159,2005-06-14 22:55:13,516
2,1160,2005-06-14 23:00:34,239
3,1161,2005-06-14 23:07:08,285
4,1162,2005-06-14 23:09:38,310
...,...,...,...
2306,3465,2005-06-21 22:10:01,510
2307,3466,2005-06-21 22:13:33,226
2308,3467,2005-06-21 22:19:25,543
2309,3468,2005-06-21 22:43:45,100


- Creamos la función rental_count_month
- Esta función contará el número de alquileres por cada customer_id en un mes y año específicos.

In [8]:
def rental_count_month(df, month, year):
    rental_count = df.groupby('customer_id').size().reset_index(name=f'rentals_{str(month).zfill(2)}_{year}')
    return rental_count

# Ejemplo de uso:
rental_count_may = rental_count_month(df_may, 5, 2005)
rental_count_june = rental_count_month(df_june, 6, 2005)

In [9]:
rental_count_may

,customer_id,rentals_05_2005
0,1,2
1,2,1
2,3,2
3,5,3
4,6,3
...,...,...
515,594,4
516,595,1
517,596,6
518,597,2


In [10]:
rental_count_june

,customer_id,rentals_06_2005
0,1,7
1,2,1
2,3,4
3,4,6
4,5,5
...,...,...
585,595,2
586,596,2
587,597,3
588,598,1


- Crear la función compare_rentals
- Esta función comparará los alquileres entre dos meses diferentes.

In [13]:
def compare_rentals(df1, df2):
    # Merge de 2 df en customer_id
    merged_df = pd.merge(df1, df2, on='customer_id', how='inner')
    
    # Calculamos la diferencia.
    month1 = df1.columns[1]
    month2 = df2.columns[1]
    merged_df['difference'] = merged_df[month2] - merged_df[month1]
    
    return merged_df

# Ejemplo:
comparison_df = compare_rentals(rental_count_may, rental_count_june)
comparison_df

,customer_id,rentals_05_2005,rentals_06_2005,difference
0,1,2,7,5
1,2,1,1,0
2,3,2,4,2
3,5,3,5,2
4,6,3,4,1
...,...,...,...,...
507,594,4,6,2
508,595,1,2,1
509,596,6,2,-4
510,597,2,3,1


- Ejecutamos y analizamos los resultados.
- Comparación de alquileres entre mayo y junio de 2005.

In [12]:
# Comparamos los alquileres entre mayo y junio.
comparison_df = compare_rentals(rental_count_may, rental_count_june)

# Resultado final.
comparison_df.head()

,customer_id,rentals_05_2005,rentals_06_2005,difference
0,1,2,7,5
1,2,1,1,0
2,3,2,4,2
3,5,3,5,2
4,6,3,4,1
